# Findings 5
Dated: 30.06.2025

# imports

In [ ]:
import sys
import os
import numpy as np

# Get the notebook's current directory
notebook_dir = os.getcwd()

# Move one level up to get the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from module import *

# computation device
This variable sets the device to be used for the Neural Network trainings:

In [ ]:
computation_device = 'auto'

# system init
This is a generic spring damper setup. Very simple and linear, with two states and one input. The state euation is the force balance of a spring damper.

In [ ]:
cstr_system = CSTR_dompc(set_seed=0)

Modifying constraints

In [ ]:
#spring_system.lbu = np.array([-10])       # [lower_bound_f_ext]
#spring_system.ubu = np.array([10])        # [upper_bound_f_ext]

# surrogate generator init
This class is designed to be totally generic, i.e., it can take any do-mpc model, or at least that is the idea.

In [ ]:
dm = DataManager(set_seed=0)

# random data

Here we generate sampled with one random initial point and random inputs. Then the data and the data is split randomly to feed different parts of the algorithm.
There is another alternate algorithm which generates data by chasing random setpoints with the help of an MPC controller.

In [ ]:
dm.random_input_sampler(system = cstr_system, n_samples=100000)
dm.data_splitter(order=1, narx_train= 0.3, cqr_train= 0.3, cqr_calibration= 0.3, test = 0.1)
#dm.data_splitter(order=2)

## data visualisation

In [ ]:
dm.visualize_data()

# NARX model

In [ ]:
dm.train_narx(hidden_layers=[10], batch_size=1000,
          learning_rate=0.1, epochs= 1000, scheduler_flag=True, device=computation_device, train_threshold=1e-4)
dm.narx.plot_narx_training_history()

# conformal quantile regression
Here qunatile regression is done to bound the errors with confidence values

In [ ]:
dm.train_cqr(alpha=0.05, hidden_layers=[10, 10],  epochs= 1000, batch_size=1000, 
             device=computation_device, train_threshold=1e-20)
dm.cqr.plot_qr_training_history()

This section visualises the quantile regression on the calibration data which the regressors has yet not seen.

In [ ]:
dm.cqr_plot_qr_error()

This plot is made against test data which till now is untouched.

In [ ]:
dm.plot_cqr_error_plotly()

# verifying simulator performance

In [ ]:
# checking simulator performance
C_a0 = 0.8 # This is the initial concentration inside the tank [mol/l]
C_b0 = 0.5 # This is the controlled variable [mol/l]
T_R0 = 134.14 #[C]
T_K0 = 130.0 #[C]

#C_a0 = 0
#C_b0 = 0
#T_R0 = 387.05
#T_J0 = 387.05

x_init = np.array([[C_a0, C_b0, T_R0, T_K0]])
dm.check_simulator(system=cstr_system, iter= 50, x_init=x_init)

## reference check
This function plots the performance of an MPC with a surrogate model.

In [ ]:
iter = 50
setpoint = None
n_horizon = 20
r = 0.5

In [ ]:
# check closed loop performance for an MPC with a surrogate model, simulated on the real system
dm.check_simulator_mpc(system=cstr_system, iter=iter, setpoint=setpoint, n_horizon=n_horizon, r=r, x_init=x_init)

# simulation

In [ ]:
# run the icb_mpc
R = 0.1*np.array([[1, 0],
              [0, 1]])
Q = 0.1*np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
tightner = 2
confidence_cutoff = 0.8
rnd_samples = 7
max_search = 10

In [ ]:
dm.check_icbmpc(system=cstr_system, iter=iter, setpoint=setpoint,
                  n_horizon=n_horizon, r=r,
                  tightner=tightner, confidence_cutoff=confidence_cutoff, 
                  rnd_samples=rnd_samples, max_search=max_search, R=R, Q=Q,
                  x_init = x_init, store_gif=True)

In [ ]:
dm.plot_simulation()

In [ ]:
dm.show_gif_matplotlib()